## References

In [55]:
# https://radimrehurek.com/gensim/tut1.html
# https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

## Prepare Notebook

In [56]:
# import packages
from gensim import corpora
import pandas as pd
import logging
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from gensim.models import Phrases
import io
import pickle

In [3]:
# download stopwords and lemmatizer from nltk package
nltk.download("stopwords")
nltk.download("wordnet")

[nltk_data] Downloading package stopwords to C:\Users\Sebastian
[nltk_data]     Birk\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Sebastian
[nltk_data]     Birk\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
# log events
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

## Load and Inspect Dataset

In [5]:
# read data with timestamp as index
tweets = pd.read_csv("tweets.csv", encoding="latin1", parse_dates=True, 
                     index_col="created", usecols=range(1,28))

In [80]:
# inspect dataframe
english_tweets.head()

,text,text_clean,favoriteCount,replyToSN,truncated,replyToSID,replyToUID,statusSource,retweetCount,longitude,...,tweetcount,movement,language3,dayofweek,weeknumber,month,idBarrio_xy,idBarrio,user,hashtags
created,,,,,,,,,,,,,,,,,,,,,
2017-11-18 19:12:28,Brew Pub to try a few of the 30 beers on offer...,Brew Pub to try a few of the 30 beers on offer...,0,NaN,False,NaN,NaN,"<a href=""http://instagram.com"" rel=""nofollow"">...",0,2.169180,...,1,1.000000,ENGLISH,Saturday,46,November,1,55,u03885,"[#olgodbarcelona, #ølgod, #triathlontraining]"
2017-11-27 10:46:20,"""Art is coming face to face with yourself"" <ed...",Art is coming face to face with yourself Par...,0,NaN,False,NaN,NaN,"<a href=""http://instagram.com"" rel=""nofollow"">...",0,2.171902,...,7,0.285714,ENGLISH,Monday,48,November,11,55,u00395,[]
2017-11-25 20:44:52,Today's quickie @ Parc de les Tres Xemeneies h...,Today's quickie Parc de les Tres Xemeneies,2,NaN,False,NaN,NaN,"<a href=""http://instagram.com"" rel=""nofollow"">...",0,2.171902,...,7,0.285714,ENGLISH,Saturday,47,November,11,55,u00395,[]
2017-11-23 10:47:18,All hail the freshest seafood in town. Let's f...,All hail the freshest seafood in town Let's fe...,0,NaN,False,NaN,NaN,"<a href=""http://instagram.com"" rel=""nofollow"">...",0,2.172170,...,3,0.666667,ENGLISH,Thursday,47,November,1,55,u01268,"[#hadthemostmusselsinmylife, #tripoftheyear]"
2017-11-19 22:30:26,Dance to the beat of your own drum.\n\n3<U+20E...,Dance to the beat of your own drum\n\n3\n\n#ur...,0,NaN,False,NaN,NaN,"<a href=""http://instagram.com"" rel=""nofollow"">...",0,2.171902,...,6,0.500000,ENGLISH,Sunday,46,November,11,55,u00495,"[#urban, #music, #live, #stage, #graffiti, #st..."


In [82]:
# display dataframe info
english_tweets.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7633 entries, 2017-11-18 19:12:28 to 2017-06-11 15:55:42
Data columns (total 28 columns):
text                    7633 non-null object
text_clean              7633 non-null object
favoriteCount           7633 non-null int64
replyToSN               150 non-null object
truncated               7633 non-null bool
replyToSID              87 non-null float64
replyToUID              150 non-null float64
statusSource            7633 non-null object
retweetCount            7633 non-null int64
longitude               7633 non-null float64
latitude                7633 non-null float64
id_seccion              7633 non-null int64
horaPeticion            7633 non-null object
id_distrito             7633 non-null int64
grupoHora               7633 non-null object
id_seccion_xy           7633 non-null int64
favoriteCountOutlier    7633 non-null int64
retweetCountOutlier     7633 non-null int64
tweetcount              7633 non-null int64
movement    

In [106]:
# describe dataframe 
tweets.describe()

,favoriteCount,replyToSID,replyToUID,retweetCount,longitude,latitude,id_seccion,id_distrito,id_seccion_xy,favoriteCountOutlier,retweetCountOutlier,tweetcount,movement,weeknumber,idBarrio_xy,idBarrio
count,23778.000000,5.740000e+02,8.210000e+02,23778.000000,23778.000000,23778.000000,2.377800e+04,23778.000000,2.377800e+04,23778.000000,23778.000000,23778.000000,23778.000000,23778.000000,23778.000000,23778.000000
mean,1.200774,9.015351e+17,4.078693e+16,0.285222,2.171064,41.395325,8.019040e+08,801903.985953,7.999476e+08,0.000673,0.000126,17.686349,0.690218,34.522962,18.136008,21.180503
std,20.403217,5.293076e+16,1.808261e+17,4.610914,0.021190,0.014936,3.028198e+03,3.023254,3.955736e+07,0.025932,0.011232,62.044925,0.326073,7.046783,21.415965,22.107374
min,0.000000,1.229677e+17,7.802900e+05,0.000000,2.059243,41.332580,8.019010e+08,801901.000000,0.000000e+00,0.000000,0.000000,1.000000,0.100000,23.000000,0.000000,1.000000
25%,0.000000,8.862349e+17,1.195479e+08,0.000000,2.159720,41.382780,8.019020e+08,801902.000000,8.019020e+08,0.000000,0.000000,1.000000,0.384615,28.000000,6.000000,6.000000
50%,0.000000,9.079774e+17,3.537928e+08,0.000000,2.174778,41.395250,8.019021e+08,801902.000000,8.019021e+08,0.000000,0.000000,3.000000,0.750000,34.000000,7.000000,9.000000
75%,1.000000,9.232741e+17,1.028215e+09,0.000000,2.176944,41.404080,8.019060e+08,801906.000000,8.019050e+08,0.000000,0.000000,8.000000,1.000000,40.000000,25.000000,31.000000
max,2449.000000,9.354775e+17,9.290872e+17,567.000000,2.226620,41.465590,8.019102e+08,801910.000000,8.019102e+08,1.000000,1.000000,440.000000,1.000000,49.000000,73.000000,73.000000


In [6]:
# divide dataset according to language: extract english language
english_tweets = tweets[tweets["language3"] == "ENGLISH"].copy()

## Data Cleaning and Preparation

### Preprocess twitter text

In [7]:
# remove links
english_tweets["text_clean"] = english_tweets["text"].str.replace(r"http\S+", "")
# remove emoticons
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"<.*>", "")
# remove @s
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"@", "")
# remove punctuation and special characters
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"&amp", "")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"\.", "")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"\,", "")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"\;", "")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"\-", "")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"\"", "")

In [8]:
# reorder columns
english_tweets.columns
cols = ['text', 'text_clean', 'favoriteCount', 'replyToSN', 'truncated', 'replyToSID',
       'replyToUID', 'statusSource', 'retweetCount', 'longitude', 'latitude',
       'id_seccion', 'horaPeticion', 'id_distrito', 'grupoHora',
       'id_seccion_xy', 'favoriteCountOutlier', 'retweetCountOutlier',
       'tweetcount', 'movement', 'language3', 'dayofweek', 'weeknumber',
       'month', 'idBarrio_xy', 'idBarrio', 'user']

english_tweets = english_tweets[cols]

#### In order to train the topic models, 3 different pooling methods for the creation of documents are used.

### LDA Training Documents Option 1 (No Pooling)

In [9]:
# treat every tweet as a document (no pooling)
documents = english_tweets["text_clean"].tolist()

### LDA Training Documents Option 2 (User Pooling)

In [10]:
# treat all tweets by one user as a document (user pooling)
user_combined = english_tweets[["text_clean","user"]].groupby("user")["text_clean"].apply(lambda x: "".join(x))
documents_user_pooling = user_combined.tolist()

### LDA Training Documents Option 3 (Hashtag Pooling)

In [11]:
# treat all tweets with same hashtags as a document (hashtag pooling)

# find all hashtags
english_tweets["hashtags"] = english_tweets["text_clean"].str.findall(r'#.*?(?=\s|$)')

# separate hashtags in columns
hashtags_tweets = pd.DataFrame(english_tweets["hashtags"].tolist(),
                               columns=["hashtag1", "hashtag2", "hashtag3", "hashtag4",
                                        "hashtag5", "hashtag6", "hashtag7", "hashtag8",
                                        "hashtag9", "hashtag10", "hashtag11", "hashtag12",
                                        "hashtag13"])

# join hashtags with tweet text
hashtags_tweets.index = english_tweets.index
hashtags_tweets = english_tweets.join(hashtags_tweets)

In [12]:
# create one dataframe with text for each hashtag column and save them in a dictionary
dict = {}
for index, item in enumerate(["hash1", "hash2", "hash3", "hash4", "hash5",
                              "hash6", "hash7", "hash8", "hash9", "hash10",
                              "hash11", "hash12", "hash13"]):
    dict[item] = hashtags_tweets[["hashtag" + str(index + 1), "text_clean"]].copy()
    dict[item].columns = ["hashtag", "text"]
    dict[item].dropna(inplace=True)

# concatenate all dataframes to one dataframe (the result is a dataframe
# where there is text for each hashtag found)
hashtags = pd.DataFrame()
for item in dict:
    hashtags = pd.concat([hashtags, dict[item]])

# combine text for each hashtag
hashtags_combined = hashtags.groupby("hashtag")["text"].apply(lambda x: "".join(x))

In [13]:
# remove some generic hashtags that cover a lot of different topics
hashtags_combined.drop(["#Barcelona", "#Catalunya", "#Spain", "#BCN", "#BARCELONA",
                        "#Espana", "#BarcelonaSpain"], inplace=True)

In [14]:
# create documents
documents_hashtag_pooling = hashtags_combined.tolist()

### Prepare Test Documents

#### The first objective of the research is to analyze the distribution of topics over the districts. For this purpose, district pooling is used to create the documents that will be tested.

In [15]:
# merge all tweets from each district (district pooling) and treat them as one document respectively
district_combined = english_tweets[["text_clean","idBarrio"]].groupby("idBarrio")["text_clean"].apply(lambda x: "".join(x))
documents_district_pooling = district_combined.tolist()

#### The second objective is to look at the dynamic topic development over time. For this purpose, the dataset is divided according to time and documents are created on this basis.

#### Divide dataframe according to month

In [59]:
# sort index
sorted_tweets = english_tweets.sort_index()

In [63]:
# check first and last date
print(sorted_tweets.index[0]) # June 2017
print(sorted_tweets.index[-1]) # December 2017

2017-06-11 13:46:35
2017-12-04 21:20:25


In [74]:
# create column that contains the month of the tweets
sorted_tweets['month'] = sorted_tweets.index.month

In [71]:
# # ignore this part!

# # split dataframe according to month
# june = sorted_tweets.loc['2017-06-01':'2017-06-30']
# july = sorted_tweets.loc['2017-07-01':'2017-07-31']
# august = sorted_tweets.loc['2017-08-01':'2017-08-31']
# september = sorted_tweets.loc['2017-09-01':'2017-09-30']
# october = sorted_tweets.loc['2017-10-01':'2017-10-31']
# november = sorted_tweets.loc['2017-11-01':'2017-11-30']
# december = sorted_tweets.loc['2017-12-01':'2017-12-31']

In [76]:
# merge all tweets from each month and treat them as one document respectively
months_combined = english_tweets[["text_clean","month"]].groupby("month")["text_clean"].apply(lambda x: "".join(x))
documents_month_pooling = months_combined.tolist()

In [78]:
# check how many documents
len(documents_month_pooling) # should be 7

7

In [91]:
# merge all tweets from one district and one month and treat them as one document respectively
districts_per_month_combined = sorted_tweets[["text_clean","month","idBarrio"]].groupby(["month","idBarrio"])["text_clean"].apply(lambda x: "".join(x))
documents_district_per_month_pooling = districts_per_month_combined.tolist()

In [92]:
# check how many documents
len(documents_district_per_month_pooling)

328

In [110]:
# inspect dataframe
districts_per_month_combined

month  idBarrio
6      1           Our last night in #Barcelona having dinner at ...
       2           Boy don't hurt your brain  Gothic Quarter Barc...
       3           Hearbeat  #boyfriend #bf #bcn #love #spain #pa...
       4           Just posted a photo  C/mònec 14 Barcelona Ramo...
       5           Un salto a la realidad #sweetdreams #bcn #summ...
       6           I was moved  inspired  My life will never be t...
       7           Another for the #Pokedex  PokemonGo Barcelona ...
       8           Special Jazz concert Sat 17th June 17  22:30  ...
       9           #suit #knife #scene #killers  Gran Via de les ...
       10          Just posted a photo  i  Barcelona mellowsheng ...
       11          Fireworks I II III by Joan Miró  Fundació Joan...
       12          CARAVANA #casualcomposition #waytosee #francba...
       14          We had a great time last week for the LouLou r...
       15          We are ready for thisOff Week 2017 is here! #m...
       16         

### Save Training and Test Documents

In [111]:
with io.open('documents.txt', 'w', encoding='utf-8') as f:
    for item in documents:
        f.write(item)
with io.open('documents_user_pooling.txt', 'w', encoding='utf-8') as f:
    for item in documents_user_pooling:
        f.write(item)
with io.open('documents_hashtag_pooling.txt', 'w', encoding='utf-8') as f:
    for item in documents_hashtag_pooling:
        f.write(item)
with io.open('documents_district_pooling.txt', 'w', encoding='utf-8') as f:
    for item in documents_district_pooling:
        f.write(item)
with io.open('documents_month_pooling.txt', 'w', encoding='utf-8') as f:
    for item in documents_month_pooling:
        f.write(item)
with io.open('documents_district_per_month_pooling.txt', 'w', encoding='utf-8') as f:
    for item in documents_district_per_month_pooling:
        f.write(item)

### Tokenize Training Documents

In [17]:
texts_no_pooling = [[word for word in document.lower().split()]
          for document in documents]

texts_user_pooling = [[word for word in document.lower().split()]
          for document in documents_user_pooling]

texts_hashtag_pooling = [[word for word in document.lower().split()]
          for document in documents_hashtag_pooling]

### Further Preprocessing of Training Documents after Tokenization

In [18]:
# remove numbers, but not words that contain numbers.
texts_no_pooling = [[token for token in doc if not token.isnumeric()] for doc in texts_no_pooling]
texts_user_pooling = [[token for token in doc if not token.isnumeric()] for doc in texts_user_pooling]
texts_hashtag_pooling = [[token for token in doc if not token.isnumeric()] for doc in texts_hashtag_pooling]

In [19]:
# remove words that are only one character.
texts_no_pooling = [[token for token in doc if len(token) > 1] for doc in texts_no_pooling]
texts_user_pooling = [[token for token in doc if len(token) > 1] for doc in texts_user_pooling]
texts_hashtag_pooling = [[token for token in doc if len(token) > 1] for doc in texts_hashtag_pooling]

In [20]:
# lemmatize all words in all documents.
lemmatizer = WordNetLemmatizer()
texts_no_pooling = [[lemmatizer.lemmatize(token) for token in doc] for doc in texts_no_pooling]
texts_user_pooling = [[lemmatizer.lemmatize(token) for token in doc] for doc in texts_user_pooling]
texts_hashtag_pooling = [[lemmatizer.lemmatize(token) for token in doc] for doc in texts_hashtag_pooling]

In [ ]:
# # ignore this part! computing bigrams did not improve models but made them worse!

# # compute bigrams
# # add bigrams and trigrams to docs (only ones that appear 5 times or more)
# bigram = Phrases(texts_no_pooling, min_count=10)
# for idx in range(len(texts_no_pooling)):
#     for token in bigram[texts_no_pooling[idx]]:
#         if '_' in token:
#             # Token is a bigram, add to document.
#             texts_no_pooling[idx].append(token)
#             
# bigram = Phrases(texts_user_pooling, min_count=10)
# for idx in range(len(texts_user_pooling)):
#     for token in bigram[texts_user_pooling[idx]]:
#         if '_' in token:
#             # Token is a bigram, add to document.
#             texts_user_pooling[idx].append(token)
#             
# bigram = Phrases(texts_hashtag_pooling, min_count=10)
# for idx in range(len(texts_hashtag_pooling)):
#     for token in bigram[texts_hashtag_pooling[idx]]:
#         if '_' in token:
#             # Token is a bigram, add to document.
#             texts_hashtag_pooling[idx].append(token)

## Save Tokenized Training Documents

In [29]:
with open('tokenized_documents_no_pooling.p', 'wb') as fp:
    pickle.dump(texts_no_pooling, fp)
    
with open('tokenized_documents_user_pooling.p', 'wb') as fp:
    pickle.dump(texts_user_pooling, fp)
    
with open('tokenized_documents_hashtag_pooling.p', 'wb') as fp:
    pickle.dump(texts_hashtag_pooling, fp)

## Refine and Vectorize Corpora

In [21]:
# define function to refine and vectorize corpus 
# (remove stopwords, very frequent and very infrequent words etc.)

# define stopwords
stpwords = 'for a of the and to in at by spain barcelona #barcelona #spain de la del en las "barcelona #bcn'.split()

def nltk_stopwords():
    return set(nltk.corpus.stopwords.words('english'))

def prep_corpus(docs, 
                additional_stopwords=set(stpwords),
                no_below=2, no_above=0.5,
                dictionary_name='tourism.dict', corpus_name='tourism.mm'):
    print('Building dictionary...')
    dictionary = corpora.Dictionary(docs)
    stopwords = nltk_stopwords().union(additional_stopwords)
    stopword_ids = map(dictionary.token2id.get, stopwords)
    dictionary.filter_tokens(stopword_ids)
    dictionary.compactify()
    dictionary.filter_extremes(no_below=no_below, no_above=no_above, keep_n=None)
    dictionary.compactify()
    dictionary.save(dictionary_name)  # store the dictionary, for future reference
    
    print('Building corpus...')
    corpus = [dictionary.doc2bow(doc) for doc in docs]
    corpora.MmCorpus.serialize(corpus_name, corpus)  # store to disk, for later use
    
    return (corpus, dictionary)

In [22]:
# run function to vectorize corpora
corpus_no_pooling = prep_corpus(texts_no_pooling,
                                dictionary_name="tourism_no_pooling.dict",
                                corpus_name="tourism_no_pooling.mm")[0]
dictionary_no_pooling = prep_corpus(texts_no_pooling,
                                    dictionary_name="tourism_no_pooling.dict",
                                    corpus_name="tourism_no_pooling.mm")[1]

corpus_user_pooling = prep_corpus(texts_user_pooling,
                                    dictionary_name="tourism_user_pooling.dict",
                                    corpus_name="tourism_user_pooling.mm")[0]
dictionary_user_pooling = prep_corpus(texts_user_pooling,
                                    dictionary_name="tourism_user_pooling.dict",
                                    corpus_name="tourism_user_pooling.mm")[1]

corpus_hashtag_pooling = prep_corpus(texts_hashtag_pooling,
                                    dictionary_name="tourism_hashtag_pooling.dict",
                                    corpus_name="tourism_hashtag_pooling.mm")[0]
dictionary_hashtag_pooling = prep_corpus(texts_hashtag_pooling,
                                    dictionary_name="tourism_hashtag_pooling.dict",
                                    corpus_name="tourism_hashtag_pooling.mm")[1]

2018-09-20 18:52:34,035 : INFO : adding document #0 to Dictionary(0 unique tokens: [])


Building dictionary...


2018-09-20 18:52:34,236 : INFO : built Dictionary(16741 unique tokens: ['#olgodbarcelona', '#triathlontraining', '#ølgod', 'beer', 'brew']...) from 7633 documents (total 79537 corpus positions)
2018-09-20 18:52:34,325 : INFO : discarding 11562 tokens: [('#olgodbarcelona', 1), ('#triathlontraining', 1), ('quickie', 1), ('#hadthemostmusselsinmylife', 1), ('#tripoftheyear', 1), ('feast!', 1), ('freshest', 1), ('hail', 1), ('2h:27m:58s', 1), ('mallan', 1)]...
2018-09-20 18:52:34,326 : INFO : keeping 5035 tokens which were in no less than 2 and no more than 3816 (=50.0%) documents
2018-09-20 18:52:34,340 : INFO : resulting dictionary: Dictionary(5035 unique tokens: ['#ølgod', 'beer', 'brew', 'offer', 'pub']...)
2018-09-20 18:52:34,345 : INFO : saving Dictionary object under tourism_no_pooling.dict, separately None
2018-09-20 18:52:34,353 : INFO : saved tourism_no_pooling.dict


Building corpus...


2018-09-20 18:52:34,622 : INFO : storing corpus in Matrix Market format to tourism_no_pooling.mm
2018-09-20 18:52:34,626 : INFO : saving sparse matrix to tourism_no_pooling.mm
2018-09-20 18:52:34,628 : INFO : PROGRESS: saving document #0
2018-09-20 18:52:34,657 : INFO : PROGRESS: saving document #1000
2018-09-20 18:52:34,690 : INFO : PROGRESS: saving document #2000
2018-09-20 18:52:34,728 : INFO : PROGRESS: saving document #3000
2018-09-20 18:52:34,761 : INFO : PROGRESS: saving document #4000
2018-09-20 18:52:34,794 : INFO : PROGRESS: saving document #5000
2018-09-20 18:52:34,822 : INFO : PROGRESS: saving document #6000
2018-09-20 18:52:34,858 : INFO : PROGRESS: saving document #7000
2018-09-20 18:52:34,883 : INFO : saved 7633x5035 matrix, density=0.108% (41550/38432155)
2018-09-20 18:52:34,886 : INFO : saving MmCorpus index to tourism_no_pooling.mm.index
2018-09-20 18:52:34,891 : INFO : adding document #0 to Dictionary(0 unique tokens: [])


...

2018-09-20 18:52:37,615 : INFO : storing corpus in Matrix Market format to tourism_hashtag_pooling.mm
2018-09-20 18:52:37,619 : INFO : saving sparse matrix to tourism_hashtag_pooling.mm
2018-09-20 18:52:37,621 : INFO : PROGRESS: saving document #0
2018-09-20 18:52:37,675 : INFO : PROGRESS: saving document #1000
2018-09-20 18:52:37,734 : INFO : PROGRESS: saving document #2000
2018-09-20 18:52:37,780 : INFO : PROGRESS: saving document #3000
2018-09-20 18:52:37,836 : INFO : PROGRESS: saving document #4000
2018-09-20 18:52:37,912 : INFO : PROGRESS: saving document #5000
2018-09-20 18:52:37,979 : INFO : PROGRESS: saving document #6000
2018-09-20 18:52:37,991 : INFO : saved 6198x9904 matrix, density=0.138% (84677/61384992)
2018-09-20 18:52:37,993 : INFO : saving MmCorpus index to tourism_hashtag_pooling.mm.index
2018-09-20 18:52:37,999 : INFO : adding document #0 to Dictionary(0 unique tokens: [])


Building dictionary...


2018-09-20 18:52:38,396 : INFO : built Dictionary(11970 unique tokens: ['#11setembre', '#2017', '#aj', '#aperitivo', '#architecture']...) from 6198 documents (total 148488 corpus positions)
2018-09-20 18:52:38,433 : INFO : discarding 1931 tokens: [('(out', 1), ('10k++)', 1), ("advisor's", 1), ('carot', 1), ('celebration', 1), ('forgotten', 1), ('garden!', 1), ('napa', 1), ('oriental', 1), ('skewer', 1)]...
2018-09-20 18:52:38,438 : INFO : keeping 9904 tokens which were in no less than 2 and no more than 3099 (=50.0%) documents
2018-09-20 18:52:38,448 : INFO : resulting dictionary: Dictionary(9904 unique tokens: ['#11setembre', '#2017', '#aj', '#aperitivo', '#architecture']...)
2018-09-20 18:52:38,455 : INFO : saving Dictionary object under tourism_hashtag_pooling.dict, separately None
2018-09-20 18:52:38,464 : INFO : saved tourism_hashtag_pooling.dict
2018-09-20 18:52:38,648 : INFO : storing corpus in Matrix Market format to tourism_hashtag_pooling.mm
2018-09-20 18:52:38,651 : INFO : s

Building corpus...


2018-09-20 18:52:38,693 : INFO : PROGRESS: saving document #1000
2018-09-20 18:52:38,756 : INFO : PROGRESS: saving document #2000
2018-09-20 18:52:38,800 : INFO : PROGRESS: saving document #3000
2018-09-20 18:52:38,835 : INFO : PROGRESS: saving document #4000
2018-09-20 18:52:38,872 : INFO : PROGRESS: saving document #5000
2018-09-20 18:52:38,914 : INFO : PROGRESS: saving document #6000
2018-09-20 18:52:38,922 : INFO : saved 6198x9904 matrix, density=0.138% (84677/61384992)
2018-09-20 18:52:38,924 : INFO : saving MmCorpus index to tourism_hashtag_pooling.mm.index


## Apply Function to Preprocess Test Documents (Before Testing Them with LDA)

#### This function has to include all the same steps that were applied to the training documents!

In [96]:
# define function
def preprocess(docs):
    ''' Conduct all preprocessing steps that are conducted to train the LDA model'''
    
    # tokenize documents
    tokenized = [[word for word in document.lower().split()]
          for document in docs]
    
    # remove words that are only one character
    tokenized = [[token for token in doc if len(token) > 1] for doc in tokenized]
    
    # lemmatize all words
    lemmatizer = WordNetLemmatizer()
    lemmatized = [[lemmatizer.lemmatize(token) for token in doc] for doc in tokenized]
    
    # define stopwords
    stpwords = 'for a of the and to in at by spain barcelona #barcelona #spain de la del en las "barcelona #bcn'.split()
    
    # get stopwords from nltk
    def nltk_stopwords():
        return set(nltk.corpus.stopwords.words('english'))

    # combine stopwords
    stopwords = nltk_stopwords().union(stpwords)
    
    # remove stopwords
    preprocessed = [[token for token in document if token not in stopwords] for document in lemmatized]
    
    return preprocessed

In [97]:
# apply function to test documents
texts_district_pooling = preprocess(documents_district_pooling)
texts_month_pooling = preprocess(documents_month_pooling)
texts_district_per_month_pooling = preprocess(documents_district_per_month_pooling)

## Save Preprocessed Test Documents

In [112]:
with open('tokenized_documents_district_pooling.p', 'wb') as fp:
    pickle.dump(texts_district_pooling, fp)
    
with open('tokenized_documents_month_pooling.p', 'wb') as fp:
    pickle.dump(texts_month_pooling, fp)
    
with open('tokenized_documents_district_per_month_pooling.p', 'wb') as fp:
    pickle.dump(texts_district_per_month_pooling, fp)

In [ ]:
# # ignore this part!

# # map tokens to ids
# print(dictionary_no_pooling.token2id)
# print(dictionary_user_pooling.token2id)
# print(dictionary_hashtag_pooling.token2id)

In [ ]:
# # ignore this part!

# # convert new document to vector 
# new_doc = "Sagrada Familia is amazing"
# new_vec_no_pooling = dictionary_no_pooling.doc2bow(new_doc.lower().split())
# print(new_vec_no_pooling)

In [ ]:
# # ignore this part! not needed for dataset!

# # corpus streaming: one document at a time
# class MyCorpus(object):
#     def __iter__(self):
#         for line in open("corpus_no_pooling.txt"):
#             # assume there's one document per line, tokens separated by whitespace
#             yield dictionary.doc2bow(line.lower().split())
#             
# corpus_memory_friendly = MyCorpus()  # doesn't load the corpus into memory!
# print(corpus_memory_friendly)
# 
# for vector in corpus_memory_friendly:  # load one vector into memory at a time
#     print(vector)